In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import DPOTrainer
import wandb

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



In [2]:
# --- Device Configuration ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# --- Model and Tokenizer Setup ---
model_name_or_path = "gpt2"
ignore_bias_buffers = False

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
if ignore_bias_buffers:
    model._ddp_params_and_buffers_to_ignore = [name for name, buffer in model.named_buffers() if buffer.dtype == torch.bool]

model_ref = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

c:\Users\silan\Desktop\A5\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# --- Dataset Loading and Prompt Extraction ---
def create_prompt(sample):
    return {"prompt": sample["input"]}


train_dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train[:90%]")
eval_dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train[90%:]")

# Extract the prompt
train_dataset = train_dataset.map(create_prompt)
eval_dataset = eval_dataset.map(create_prompt)

Map:   0%|          | 0/11573 [00:00<?, ? examples/s]

Map:   0%|          | 0/1286 [00:00<?, ? examples/s]

In [5]:
# --- Hyperparameters ---
learning_rate = 0.0000538
per_device_train_batch_size = 8
gradient_accumulation_steps = 4
max_length = 512
max_prompt_length = 256
max_target_length = 256
max_steps = 500
report_to = "wandb"
gradient_checkpointing = True
beta = 0.2

In [6]:
# --- Weights & Biases Setup ---
wandb.init(
    project="dpo-training",
    config={
        "model_name": model_name_or_path,
        "dataset": "argilla/distilabel-intel-orca-dpo-pairs",
        "learning_rate": learning_rate,
        "batch_size": per_device_train_batch_size,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "max_length": max_length,
        "max_prompt_length": max_prompt_length,
        "max_target_length": max_target_length,
        "max_steps": max_steps,
        "beta": beta,
        "gradient_checkpointing": gradient_checkpointing,
        "optimizer": "adamw_torch",
    },
)

# --- Training Arguments ---
training_args = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    max_steps=max_steps,
    remove_unused_columns=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    evaluation_strategy="steps",
    logging_first_step=True,
    logging_steps=5,
    eval_steps=50,
    output_dir="./nlp-a5",
    optim="adamw_torch",
    warmup_steps=50,
    report_to=report_to,
    bf16=True,
    gradient_checkpointing=gradient_checkpointing,
    save_strategy="steps",
    save_steps=50,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sila-nmht (sila-nmht-asian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


c:\Users\silan\Desktop\A5\.venv\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# --- DPOTrainer Initialization ---
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=beta,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    max_length=max_length,
    max_target_length=max_target_length,
    max_prompt_length=max_prompt_length,
    generate_during_eval=True,
)

Map:   0%|          | 0/11573 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1105 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1286 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
# --- Training ---
dpo_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/500 [00:00<?, ?it/s]

c:\Users\silan\Desktop\A5\.venv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\silan\Desktop\A5\.venv\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6931, 'grad_norm': 86.32217407226562, 'learning_rate': 1.076e-06, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -429.841064453125, 'logps/chosen': -380.14141845703125, 'logits/rejected': -101.80658721923828, 'logits/chosen': -101.73731994628906, 'epoch': 0.0}
{'loss': 0.6917, 'grad_norm': 82.48517608642578, 'learning_rate': 5.38e-06, 'rewards/chosen': -0.655906617641449, 'rewards/rejected': -1.1794593334197998, 'rewards/accuracies': 0.515625, 'rewards/margins': 0.5235527753829956, 'logps/rejected': -413.0698547363281, 'logps/chosen': -359.45428466796875, 'logits/rejected': -104.19410705566406, 'logits/chosen': -102.56261444091797, 'epoch': 0.01}
{'loss': 0.4781, 'grad_norm': 51.92380142211914, 'learning_rate': 1.076e-05, 'rewards/chosen': -0.745120644569397, 'rewards/rejected': -2.0767862796783447, 'rewards/accuracies': 0.8125, 'rewards/margins': 1.3316656351089478, 'logps/rejected': -422.7705078125, 'log

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\silan\Desktop\A5\.venv\lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


...

c:\Users\silan\Desktop\A5\.venv\lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'game_log': <wandb.sdk.data_types.table.Table object at 0x000002474D813F70>, 'epoch': 1.38}


  0%|          | 0/161 [00:00<?, ?it/s]

{'eval_loss': 0.6408655047416687, 'eval_runtime': 112.62, 'eval_samples_per_second': 11.419, 'eval_steps_per_second': 1.43, 'eval_rewards/chosen': 0.9778485894203186, 'eval_rewards/rejected': -2.149104356765747, 'eval_rewards/accuracies': 0.8234990239143372, 'eval_rewards/margins': 3.126952648162842, 'eval_logps/rejected': -410.6468505859375, 'eval_logps/chosen': -337.3829040527344, 'eval_logits/rejected': -66.98162841796875, 'eval_logits/chosen': -67.84807586669922, 'epoch': 1.38}
{'train_runtime': 4061.3209, 'train_samples_per_second': 3.94, 'train_steps_per_second': 0.123, 'train_loss': 0.5710406829714775, 'epoch': 1.38}


TrainOutput(global_step=500, training_loss=0.5710406829714775, metrics={'train_runtime': 4061.3209, 'train_samples_per_second': 3.94, 'train_steps_per_second': 0.123, 'total_flos': 0.0, 'train_loss': 0.5710406829714775, 'epoch': 1.38217000691085})

In [9]:
# --- Saving ---
dpo_trainer.save_model("./trained_model_orca_dpo_wandb")

In [10]:
# --- Finish wandb run ---
wandb.finish()

eval/logits/chosen,▁▃▅▆█████▇
eval/logits/rejected,▁▃▅▆█████▇
eval/logps/chosen,▄▃▁▁▅▅█▄█▇
eval/logps/rejected,█▅▃▁▅▄█▂▆▅
eval/loss,█▇▅▆▃▁▁▂▃▂
eval/rewards/accuracies,▁▃▃▅▄▆▄█▆█
eval/rewards/chosen,▄▃▁▁▅▅█▄█▇
eval/rewards/margins,▁▄▅▇▅▇▄█▇▇
eval/rewards/rejected,█▅▃▁▅▄█▂▆▅
eval/runtime,▅▅▅▇█▃▅▅▃▁
eval/samples_per_second,▃▄▄▂▁▅▄▄▅█


In [ ]:
from huggingface_hub import login

login()
dpo_trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/silanm/nlp-a5/commit/9c2eccfc0a7c23c6f8e54e323523296a810202a6', commit_message='End of training', commit_description='', oid='9c2eccfc0a7c23c6f8e54e323523296a810202a6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/silanm/nlp-a5', endpoint='https://huggingface.co', repo_type='model', repo_id='silanm/nlp-a5'), pr_revision=None, pr_num=None)